# Import

In [3]:
# update tensorflow to 2.11.0
!python3 -m pip install tensorflow
!python3 -m pip install finta
!python3 -m pip install -q --upgrade keras-nlp
!python3 -m pip install numpy
!python3 -m pip install pandas
!python3 -m pip install matplotlib
!pip install plotly==5.3.1
!python3 -m pip install opencv-python
!python3 -m pip install -U kaleido


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.0/289.0 kB 1.9 MB/s eta 0:00:00a 0:00:01


In [5]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K
import pandas as pd
# import scipy.signal
import time
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from PIL import Image
import cv2
import io
from finta import TA
# import keras_nlp
import random
import os
import threading
# from s3fs.core import S3FileSystem
# s3 = S3FileSystem()

np.random.seed(0)
tf.random.set_seed(0)

print(tf.version.VERSION)

ImportError: cannot import name 'AlbertClassifier' from partially initialized module 'keras_nlp.src.models.albert.albert_classifier' (most likely due to a circular import) (/Users/joshuaattridge/Library/Python/3.9/lib/python/site-packages/keras_nlp/src/models/albert/albert_classifier.py)

# Gather Training Data
classes:
0 - no trend
1 - short trend
2 - long trend

In [ ]:
candle_window_size = 200
timeout_limit = 100
# stoploss = 0.005
risk_to_reward = 2

def render(candle_data,SLDistance):
    fig = go.Figure()
    fig.add_trace(
        go.Candlestick(
            open=candle_data["open"],
            high=candle_data["high"],
            low=candle_data["low"],
            close=candle_data["close"], 
        )
    )

    # the candles are too thick for the image
    fig.update_traces(
        line=dict(width=1),
        selector=dict(type="candlestick")
    )

    # make the background white
    fig.update_layout(
        plot_bgcolor="white",
    )

    # remove the legend
    fig.update_layout(showlegend=False)

    # add the stoploss lines
    fig.add_shape(
        dict(
            type="line",
            x0=candle_window_size-1,
            y0=candle_data["close"].iloc[-1] + SLDistance,
            x1=candle_window_size,
            y1=candle_data["close"].iloc[-1] + SLDistance,
            line=dict(color="black", width=1),
        )
    )
    fig.add_shape(
        dict(
            type="line",
            x0=candle_window_size-1,
            y0=candle_data["close"].iloc[-1] - SLDistance,
            x1=candle_window_size,
            y1=candle_data["close"].iloc[-1] - SLDistance,
            line=dict(color="black", width=1),
        )
    )

    # the graph isnt going up to the edge of the image so make it go to the edge
    fig.update_xaxes(range=[0, candle_window_size])

    # remove small graph at the bottom
    fig.update_layout(
        xaxis=dict(
            rangeslider=dict(
                visible=False,
            ),
        )
    )
    # remove white space around the graph
    fig.update_layout(margin=dict(l=0, r=0, t=0, b=0))
    # remove the x and y axis
    fig.update_xaxes(visible=False)
    fig.update_yaxes(visible=False)

    # lower the resolution of the image to 420p
    fig.update_layout(width=len(candle_data), height=50)

    # convert the plotly figure to a numpy array
    fig_bytes = fig.to_image(format="png")
    buf = io.BytesIO(fig_bytes)
    img = Image.open(buf)
    img = np.asarray(img)
    # convert the image to grayscale
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)


    # if the colors in the image are not white then make them black
    img[img < 255] = 0

    # save the image
    cv2.imwrite("render.png", img)

    # convert the image to bit data 1 or 0 instead of 0 or 255 then set the data type to boolean
    img = img / 255
    img = img.astype(np.int8)

    return img

# load data from data.npy
try:
    print("trying to load data from file")
    data = np.load('data.npy', allow_pickle=True)
    results = np.load('results.npy', allow_pickle=True)

    # only use the first 10000 data points
    data = data[:10000]
    results = results[:10000]

except:
    try:
        print("trying to load data from s3")
        data_location = 's3://sagemakerforexaidata/data.npy'
        results_location = 's3://sagemakerforexaidata/results.npy'
        data = np.load(s3.open(data_location), allow_pickle=True)
        results = np.load(s3.open(results_location), allow_pickle=True)
    except Exception as e:
        print("failed to load data creating new data")
        data = []
        results = []

# if data is empty, create a new one
if len(data) == 0 or len(results) == 0:
    # for each file in the data folder
    cwd = os.getcwd()
    path = os.path.join(cwd, 'data')
    files = os.listdir(path)

    for file in files:
        chart_data = pd.read_csv(os.path.join(path, file))

        # chart_data = chart_data.iloc[:3000].reset_index(drop=True) # remove this line to use the whole dataset
        atr_multiplier = 1.5
        chart_data['SLDistance'] = round(TA.ATR(chart_data, 14) * atr_multiplier,5)
        chart_data.drop(['time'], axis=1, inplace=True)
        chart_data = chart_data.iloc[500:].reset_index(drop=True)

        def get_data_and_trend(candle_pos):
            if candle_pos + timeout_limit > len(chart_data):
                return

            window_candle_data = chart_data[candle_pos-candle_window_size:candle_pos]
            starting_price = window_candle_data['close'].iloc[-1]
            SLDistance = window_candle_data['SLDistance'].iloc[-1]
            hit_short_SL = False
            hit_short_TP = False
            hit_long_SL = False
            hit_long_TP = False
            for i in range(candle_pos+1, candle_pos + timeout_limit):
                if chart_data['high'].iloc[i] > starting_price + SLDistance:
                    hit_short_SL = True
                if chart_data['low'].iloc[i] < starting_price - (SLDistance * risk_to_reward):
                    hit_short_TP = True
                    break
                if chart_data['low'].iloc[i] < starting_price - SLDistance:
                    hit_long_SL = True
                if chart_data['high'].iloc[i] > starting_price + (SLDistance * risk_to_reward):
                    hit_long_TP = True
                    break
            
            trend = 0
            if hit_short_SL == False and hit_short_TP == True:
                # short trend
                trend = 1
            if hit_long_SL == False and hit_long_TP == True:
                # long trend
                trend = 2

            rendered_data = render(window_candle_data, SLDistance)
            data.append(rendered_data)
            results.append(trend)

            # add more data by flipping the rendered data upside down and reversing the trend
            rendered_data = np.flip(rendered_data, 0)
            if trend == 1:
                trend = 2
            elif trend == 2:
                trend = 1
            data.append(rendered_data)
            results.append(trend)
        
        # run get_data_and_trend in parallel using threads
        data_collection_threads = []
        for candle_pos in range(candle_window_size, len(chart_data)):
            try:
                data_collection_threads.append(
                    threading.Thread(target=get_data_and_trend, args=(candle_pos,))
                )
                data_collection_threads[-1].start()
                progress = round((candle_pos / len(chart_data)) * 100, 2)
                print("Thread Progress: " + str(progress) + "%", end="\r")
            # if failed to start a new thread then wait for all threads to finish and try again make sure to remove the threads that have finished
            except:
                for thread in data_collection_threads:
                    # if thread has started then join it
                    if thread.is_alive():
                        thread.join()
                data_collection_threads = []
                data_collection_threads.append(
                    threading.Thread(target=get_data_and_trend, args=(candle_pos,))
                )
                data_collection_threads[-1].start()
                progress = round((candle_pos / len(chart_data)) * 100, 2)
                print("Thread Progress: " + str(progress) + "%", end="\r")

        # wait for all threads to finish
        for thread in data_collection_threads:
            if thread.is_alive():
                thread.join()
        data_collection_threads = []

        # create a text based progress bar that always stays on the same line and updates with percentage complete this must include both loops
        progress = round((candle_pos / len(chart_data)) * 100, 2)
        print("Progress: " + str(progress) + "%", end="\r")
            

    data = np.array(data)
    results = np.array(results)

    # write the data to a file npy file that is zip compressed
    np.save('data.npy', data, allow_pickle=True)
    np.save('results.npy', results, allow_pickle=True)

unique, counts = np.unique(results, return_counts=True)
count_dict = dict(zip(unique, counts))

print("Results:")
print(count_dict)

# remove some of the data and results at random so all of the results have equal amounts
min_count = min(counts)
remove0 = count_dict[0] - min_count
remove1 = count_dict[1] - min_count
remove2 = count_dict[2] - min_count

index_remove = []
while remove0 > 0 or remove1 > 0 or remove2 > 0:
    print("Removing: " + str(remove0) + " " + str(remove1) + " " + str(remove2), end="\r")
    index = random.randint(0, len(results)-1)
    if index not in index_remove:
        if results[index] == 0 and remove0 > 0:
            index_remove.append(index)
            remove0 -= 1
        if results[index] == 1 and remove1 > 0:
            index_remove.append(index)
            remove1 -= 1
        if results[index] == 2 and remove2 > 0:
            index_remove.append(index)
            remove2 -= 1

data = np.delete(data, index_remove, axis=0)
results = np.delete(results, index_remove, axis=0)

# print the lenths of the data and results
print("Data Length: " + str(len(data)))
print("Results Length: " + str(len(results)))

unique, counts = np.unique(results, return_counts=True)
count_dict = dict(zip(unique, counts))

print("Results:")
print(count_dict)

# split the data into training and testing 30% testing and delete the original data to save memory
training_data = data[:round(len(data)*0.7)]
training_results = results[:round(len(results)*0.7)]
testing_data = data[round(len(data)*0.7):]
testing_results = results[round(len(results)*0.7):]
del data


# Classification

In [ ]:
# create an advanced model that can predict forex chart trends based on the rendered images
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(len(training_data[0]),len(training_data[0][0]), 1)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.25),
    tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.Conv2D(128, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.Conv2D(256, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.Conv2D(512, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(1,1), # decrease the size of the pooling window
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(3, activation='softmax')
])

# create a custom metric to calculate the win rate of the model
def win_rate(y_true, y_pred):
    # convert y_pred to the max value of the array this needs to be done because the model returns an array of probabilities
    y_pred = tf.math.argmax(y_pred, axis=1)

    # convert the values to float32
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)

    # calculate the win rate of the model if the prediction is equal to the result and not equal to 0 then it is a win otherwise it is a loss
    win_rate = tf.math.reduce_mean(tf.where(tf.math.logical_and(tf.math.equal(y_true, y_pred), tf.math.not_equal(y_pred, 0)), 1.0, 0.0))
    return win_rate

def forex_loss(y_true, y_pred):

    loss = tf.keras.losses.sparse_categorical_crossentropy(y_true, y_pred)

    return loss

# create a adam optimizer with a custom learning rate 
adam = tf.keras.optimizers.Adam(learning_rate=0.00001)

model.compile(loss=forex_loss, optimizer=adam, metrics=[win_rate]) #remove accuracy metric on next run
model.summary()

early_stopping_val_win_rate = tf.keras.callbacks.EarlyStopping(monitor='win_rate', mode='max', patience=10, restore_best_weights=True)

# create some class weights so guessing class 1 and 2 correctly is more important than guessing class 0 correctly
class_weight = {0: 0.5, 1: 1, 2: 1}

# train the model
model.fit(training_data, training_results, epochs=1000, batch_size=128, validation_data=(testing_data, testing_results), callbacks=[early_stopping_val_win_rate])

# save the model
model.save("model_" + str(round(max(model.history.history['val_win_rate']) * 100, 2)) + "_winrate.h5")

# plot the training and validation win rate
plt.plot(model.history.history['win_rate'])
plt.plot(model.history.history['val_win_rate'])
plt.title('Model Win Rate')
plt.ylabel('Win Rate')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()
# write plot to file
plt.savefig("model_" + str(round(max(model.history.history['val_win_rate']) * 100, 2)) + "_winrate.png", format='png')

# plot the training and validation loss
plt.plot(model.history.history['loss'])
plt.plot(model.history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()
# write plot to file
plt.savefig("model_" + str(round(max(model.history.history['val_win_rate']) * 100, 2)) + "_loss.png", format='png')

# Test the model

In [ ]:
# test the model on the testing data
test_loss, test_win_rate = model.evaluate(testing_data, testing_results, verbose=2)
print("Test Loss: " + str(test_loss))
print("Test Win Rate: " + str(test_win_rate))

# make a prediction for each of the testing data
predictions = model.predict(np.array(testing_data))
# print the first 10 predictions
print("Predictions:")
for i in range(10):
    print(predictions[i])

# compare the predictions to the actual results for each class (no trend, short trend, long trend) and display the results

win_confidence_tracker = []
loss_confidence_tracker = []

classes = ["No Trend", "Short Trend", "Long Trend"]
for i in range(1,3):
    for j in range(len(predictions)):
        if np.argmax(predictions[j]) == i:
            if np.argmax(predictions[j]) == testing_results[j]:
                win_confidence_tracker.append(predictions[j][i])
            else:
                loss_confidence_tracker.append(predictions[j][i])

# print the amount of times the model picked a class
print("No Trend: " + str(len([x for x in predictions if np.argmax(x) == 0])))
print("Short Trend: " + str(len([x for x in predictions if np.argmax(x) == 1])))
print("Long Trend: " + str(len([x for x in predictions if np.argmax(x) == 2])))


# print the win rate
print("Win Rate: " + str(round(len(win_confidence_tracker) / (len(win_confidence_tracker) + len(loss_confidence_tracker)) * 100, 2)) + "%")
# print risk to reward ratio
print("Risk to Reward Ratio: " + str(risk_to_reward))
# print the highest confidence of the correct predictions
print("Highest Correct Confidence: " + str(round(max(win_confidence_tracker) * 100, 2)) + "%")
# print the lowest confidence of the correct predictions
print("Lowest Correct Confidence: " + str(round(min(win_confidence_tracker) * 100, 2)) + "%")
# print the highest incorrect confidence
print("Highest Incorrect Confidence: " + str(round(max(loss_confidence_tracker) * 100, 2)) + "%")
# print the lowest incorrect confidence
print("Lowest Incorrect Confidence: " + str(round(min(loss_confidence_tracker) * 100, 2)) + "%")

plt.hist(win_confidence_tracker, bins=100, alpha=0.5, label='Correct')
plt.hist(loss_confidence_tracker, bins=100, alpha=0.5, label='Incorrect')
plt.legend(loc='upper right')
# write plot to file
plt.savefig("model_" + str(round(max(model.history.history['val_win_rate']) * 100, 2)) + "_confidence.png", format='png')

